In [ ]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import math

In [ ]:
pickle_file = 'indoor_outdoor_1.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)



In [ ]:
batch_size=128
hidden_units=[1024]
IMAGE_PIXELS=1000
NUM_CLASSES=train_labels.shape[1]
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

   

graph=tf.Graph()
with graph.as_default():
    tf_train_dataset=tf.placeholder(tf.float32,shape=(batch_size,IMAGE_PIXELS))
    tf_train_labels=tf.placeholder(tf.float32,shape=(batch_size,NUM_CLASSES))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    def inference(images1,images2,images3,hidden_units):
        l=0
        N=len(hidden_units)
        weights=[]
        biases=[]
        for h in range(N+1):
            a=IMAGE_PIXELS if h==0 else hidden_units[h-1]
            b=NUM_CLASSES if h==N else hidden_units[h]
            weights.append(tf.Variable(
                     tf.truncated_normal([a, b],
                                    stddev=1.0 / math.sqrt(float(IMAGE_PIXELS))),
                name='weights'+str(h)))
            biases.append(tf.Variable(tf.zeros([b]),name='biases'+str(h)))
            l=l+tf.nn.l2_loss(weights[h])
        input_1=images1
        input_2=images2
        input_3=images3
       
        for h in range(N):
            input_1= tf.nn.relu(tf.matmul(input_1, weights[h]) + biases[h])
            input_1=tf.nn.dropout(input_1,0.5)
            input_2= tf.nn.relu(tf.matmul(input_2, weights[h]) + biases[h])
            input_3= tf.nn.relu(tf.matmul(input_3, weights[h]) + biases[h])
        logits1 = tf.matmul(input_1, weights[N]) + biases[N]
        logits2 = tf.matmul(input_2, weights[N]) + biases[N]
        logits3 = tf.matmul(input_3, weights[N]) + biases[N]
        #saver=tf.train.Saver()
        return logits1,logits2,logits3,l
  
   
    logits1,logits2,logits3,l=inference(tf_train_dataset,tf_valid_dataset,tf_test_dataset,hidden_units)
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits1,tf_train_labels))+0.0001*l
    #global_step=tf.Variable(0)
    #learning_rate=tf.train.exponential_decay(0.5,global_step,100,0.95)
    #optimizer=tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)+0.0001*l
    optimizer=tf.train.AdagradOptimizer(1.0).minimize(loss)
    train_prediction=tf.nn.softmax(logits1)
    valid_prediction=tf.nn.softmax(logits2)
    test_prediction=tf.nn.softmax(logits3)


num_steps=3000
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print ("Initialized")
    for step in range(num_steps):
        offset=(step*batch_size)%(train_labels.shape[0]-batch_size)
        batch_data=train_dataset[offset:(offset + batch_size),:]
        batch_labels=train_labels[offset:(offset + batch_size)]
        feed_dict={tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
   
        if (step % 100 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
            #saver.save(session,'model.ckpt')
    temp=test_prediction.eval()
    n=0
    for i in range(temp.shape[0]):
        
         n=int(max(temp[i,:])>0.5)+n
    print(n*1.0/temp.shape[0])
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    
    #83.3    